In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import re

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
def get_files(path):
    import os
    files = os.listdir(path)
    return [os.path.join(path, file) for file in files if file.endswith('.txt')]

In [8]:
from neo4j import GraphDatabase

# 连接到 Neo4j
URI = "neo4j+s://fed64bff.databases.neo4j.io"  # 修改为你的 Neo4j 地址
AUTH = ("neo4j", "O8tsGxq0Zr-2rTV3CckvG5tflwyXeWHt6tHvws0ziow")  # 修改为你的用户名和密码

# 属于一个节的所有主要概念
def get_chapter_full_subgraph():
    query = """
    MATCH (chapter:`节`)
    CALL (chapter) {
        OPTIONAL MATCH path = (chapter)-[*]->(sub)
        RETURN collect({node: sub, relationships: relationships(path)}) AS subgraph
    }
    RETURN chapter.name AS chapter_title, chapter.id AS chapter_id, subgraph;
    """

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run(query)
            chapters = []
            for record in result:
                chapter_info = {
                    "chapter_title": record["chapter_title"],
                    "chapter_id": record["chapter_id"],
                    "subgraph": []
                }
                # print(record)
                for entry in record["subgraph"]:
                    node = entry["node"]
                    relationships = entry["relationships"]
                    chapter_info["subgraph"].append({
                        "node": dict(node),  # 获取所有节点属性
                        "relationships": [{"type": rel.type, "start": rel.start_node["id"], "end": rel.end_node["id"]} for rel in relationships]
                    })
                chapters.append(chapter_info)
            return chapters

# 获取所有章的完整子图
full_subgraphs = get_chapter_full_subgraph()

# 打印结果
import json
print(json.dumps(full_subgraphs, indent=4, ensure_ascii=False))

[
    {
        "chapter_title": "第一节 教育概述",
        "chapter_id": 3004,
        "subgraph": [
            {
                "node": {
                    "chapter": "第一节 教育概述",
                    "level": 0,
                    "name": "教育的类型 【单选、填空、名词解释、辨析】",
                    "description": "",
                    "page_content_sha1": "da39a3ee5e6b4b0d3255bfef95601890afd80709",
                    "page_content": "",
                    "id": 357
                },
                "relationships": [
                    {
                        "type": "包含",
                        "start": null,
                        "end": 357
                    }
                ]
            },
            {
                "node": {
                    "chapter": "第一节 教育概述",
                    "level": 1,
                    "name": "学龄教育与成人教育",
                    "description": "\n根据受教育者的年龄阶段，教育可以分为学龄教育和成人教育。",
                    "page_content_sha1": "336c5db4df6821b370a0f43158558563c

In [14]:
def get_relation_from_text_v2(main_concepts):
    decs_sys_template = """
-目标-
根据主要概念列表，依赖各概念的描述，提取**同级概念**之间的关系，确保逻辑清晰、便于教学使用。

-输入形式-
主要概念列表：包括概念名称、编号、描述。

-输出关系列表-
每个关系以如下格式表示：
<<编号1:概念名称1{tuple_delimiter}关系类型{tuple_delimiter}编号2:概念名称2{tuple_delimiter}关系描述{tuple_delimiter}权重(置信度，1-10，表示关系的可信度，越高越可信)>>
示例：
<<1001:教育心理学{tuple_delimiter}相似关系{tuple_delimiter}1002:发展心理学{tuple_delimiter}二者都研究人的心理发展，但前者关注教育环境中的心理现象{tuple_delimiter}8>>
<<2:加法{tuple_delimiter}前驱关系{tuple_delimiter}3:乘法{tuple_delimiter}学会加法是学习乘法的前置条件{tuple_delimiter}9>>
<<24:1903年的“癸卯学制”{tuple_delimiter}发展演变关系{tuple_delimiter}25:1912-1913年的“壬子癸丑学制”{tuple_delimiter}1912-1913年的“壬子癸丑学制”是在1903年的“癸卯学制”基础上进行的改革{tuple_delimiter}7>>

关系类型包括：
-发展演变关系：概念1发展演变为概念2（通常在时间上有明显顺序）
-相似关系：两个概念具有相似性
-影响关系：概念2受概念1的影响
-前驱关系：学会概念1是学习概念2的前置条件，概念2依赖于概念1

-注意事项-
- **关系必须符合教学逻辑**，确保教师在讲解时能合理使用。
- 关系类型必须符合要求
######################
"""
    decs_sys_prompt = SystemMessagePromptTemplate.from_template(decs_sys_template)

    decs_human_template="""
    ######################
    -真实数据- 
    主要概念列表：{input_text} 
    分隔符：{tuple_delimiter}
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([decs_sys_prompt, decs_human_template])
    llm = ChatOpenAI(
        api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
        base_url="https://api.chatanywhere.tech/v1",
        model="gpt-4o",
    )
    chain = LLMChain(llm=llm, prompt=chat_prompt)

    ans = ""
    for level in range(1, 4):
        tmp = [item for item in main_concepts if item[2] == level]
        main_concepts_str = ""
        for name, id, level, decs in tmp:
            main_concepts_str += f"{name}, {id}, {level}, {decs}\n"
        if main_concepts_str != "":
            answer = chain.invoke({
                "input_text": main_concepts_str,
                "tuple_delimiter": '##'
                })
            ans += answer['text']
        
    return ans

In [ ]:
for subgraph in full_subgraphs:
    chapter_title = subgraph["chapter_title"]
    main_concepts = []
    for entry in subgraph["subgraph"]:
        node = entry["node"]
        relationships = entry["relationships"]
        
        main_concepts.append((node["name"], node["id"], node['level'], node["description"]))
        # print(node["name"], node["id"], node['level'], node["description"])

    # 保存主要概念到json
    # with open("D:\\code\\agent_demo\\kg\\main_concept\\" + chapter_title + ".json", "w", encoding="utf-8") as f:
    #     json.dump(main_concepts, f, indent=4, ensure_ascii=False)
    ans = get_relation_from_text_v2(main_concepts)
    with open("D:\\code\\agent_demo\\kg\\relation\\" + chapter_title + ".txt", "w", encoding="utf-8") as f:
        f.write(ans)

In [ ]:
def parse_relationships(file_path):
    relationships = []
    
    # 读取 TXT 文件
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.read()
    
    # 正则表达式解析格式
    pattern = re.compile(r"<<(\d+):([^{}]+)##([^{}]+)##(\d+):([^{}]+)##([^>>]+)>>")
    
    matchs = pattern.search(lines)
    
    for match in matchs:
        concept1_id = int(match.group(1))  # 编号1
        concept1_name = match.group(2)  # 名称1
        relation_type = match.group(3)  # 关系类型
        concept2_id = int(match.group(4))  # 编号2
        concept2_name = match.group(5)  # 名称2
        relation_desc = match.group(6)  # 关系描述

        relationships.append((concept1_id, concept1_name, relation_type, concept2_id, concept2_name, relation_desc))
    
    return relationships

def update_neo4j_relationships(file_path):
    relationships = parse_relationships(file_path)

    query = """
    MATCH (c1:`主要概念` {id: $id1})
    MATCH (c2:`主要概念` {id: $id2})
    MERGE (c1)-[r:RELATION {type: $relation_type}]->(c2)
    SET r.description = $relation_desc
    """

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            for id1, name1, relation_type, id2, name2, relation_desc in relationships:
                session.run(query, id1=id1, id2=id2, relation_type=relation_type, relation_desc=relation_desc)

    print("✅ 关系已成功更新到 Neo4j！")

# 运行代码
# file_path = r"D:\code\agent_demo\kg\relation\第二节 德育的内容与原则.txt"
# update_neo4j_relationships(file_path)

file_floder = r"D:\code\agent_demo\kg\relation"
files = get_files(file_floder)
for file in files:
    update_neo4j_relationships(file)